In [1]:
!pip install -U pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("dataset.csv")
print("Shape of dataset:", df.shape)
print("\nColumns:\n", df.columns.tolist())
df.head()

Shape of dataset: (18368, 12)

Columns:
 ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [4]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df.drop(['LastUpdatedDate', 'LastUpdatedTime'], axis=1, inplace=True)

df = df.sort_values(by='Timestamp')

print(df[['SystemCodeNumber', 'Timestamp']].head())

      SystemCodeNumber           Timestamp
0          BHMBCCMKT01 2016-10-04 07:59:00
5248       BHMNCPHST01 2016-10-04 07:59:00
3936       BHMMBMMBX01 2016-10-04 07:59:00
6560       BHMNCPNST01 2016-10-04 07:59:00
17056         Shopping 2016-10-04 07:59:00


In [5]:
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df[['SystemCodeNumber', 'Occupancy', 'Capacity', 'OccupancyRate']].head()

,SystemCodeNumber,Occupancy,Capacity,OccupancyRate
0,BHMBCCMKT01,61,577,0.105719
5248,BHMNCPHST01,237,1200,0.197500
3936,BHMMBMMBX01,264,687,0.384279
6560,BHMNCPNST01,249,485,0.513402
17056,Shopping,614,1920,0.319792


**MODEL 1**

In [6]:
def baseline_price_model(df, alpha=5, base_price=10):
    df = df.copy()
    df['Price'] = None

    for lot, group in df.groupby('SystemCodeNumber'):
        price = base_price
        indices = group.index

        for i in indices:
            df.at[i, 'Price'] = price
            price = price + alpha * df.at[i, 'OccupancyRate']

    df['Price'] = df['Price'].astype(float)
    return df


In [7]:
df_model1 = baseline_price_model(df)
df_model1[['SystemCodeNumber', 'Timestamp', 'OccupancyRate', 'Price']].head(10)


,SystemCodeNumber,Timestamp,OccupancyRate,Price
0,BHMBCCMKT01,2016-10-04 07:59:00,0.105719,10.0
5248,BHMNCPHST01,2016-10-04 07:59:00,0.197500,10.0
3936,BHMMBMMBX01,2016-10-04 07:59:00,0.384279,10.0
6560,BHMNCPNST01,2016-10-04 07:59:00,0.513402,10.0
17056,Shopping,2016-10-04 07:59:00,0.319792,10.0
2624,BHMEURBRD01,2016-10-04 07:59:00,0.248936,10.0
7872,Broad Street,2016-10-04 07:59:00,0.257971,10.0
14432,Others-CCCPS8,2016-10-04 07:59:00,0.336611,10.0
9184,Others-CCCPS105a,2016-10-04 07:59:00,0.352912,10.0
10496,Others-CCCPS119a,2016-10-04 07:59:00,0.069568,10.0


In [8]:
df_model1[df_model1['SystemCodeNumber'] == 'BHMBCCMKT01'][['Timestamp', 'OccupancyRate', 'Price']].head(10)


,Timestamp,OccupancyRate,Price
0,2016-10-04 07:59:00,0.105719,10.000000
1,2016-10-04 08:25:00,0.110919,10.528596
2,2016-10-04 08:59:00,0.138648,11.083189
3,2016-10-04 09:32:00,0.185442,11.776430
4,2016-10-04 09:59:00,0.259965,12.703640
5,2016-10-04 10:26:00,0.306759,14.003466
6,2016-10-04 10:59:00,0.379549,15.537262
7,2016-10-04 11:25:00,0.428076,17.435009
8,2016-10-04 11:59:00,0.448873,19.575390
9,2016-10-04 12:29:00,0.461005,21.819757


**MODEL 2**

In [9]:
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
df_model1['TrafficLevel'] = df_model1['TrafficConditionNearby'].map(traffic_map)
vehicle_map = {
    'bike': 0.5,
    'car': 1.0,
    'truck': 1.5,
    'cycle': 0.3
}
df_model1['VehicleWeight'] = df_model1['VehicleType'].map(vehicle_map)
df_model1[['VehicleType', 'VehicleWeight']].drop_duplicates()
df_model1[['TrafficConditionNearby', 'TrafficLevel', 'VehicleType', 'VehicleWeight']].head()


,TrafficConditionNearby,TrafficLevel,VehicleType,VehicleWeight
0,low,0.0,car,1.0
5248,low,0.0,bike,0.5
3936,low,0.0,car,1.0
6560,low,0.0,car,1.0
17056,low,0.0,cycle,0.3


In [12]:
def calculate_demand_score(df):
    alpha1 = 0.4
    alpha2 = 0.2
    alpha3 = 0.1
    alpha4 = 0.2
    alpha5 = 0.1

    df['DemandScore'] = (
        alpha1 * df['OccupancyRate'] +
        alpha2 * df['QueueLength'] +
        alpha3 * df['IsSpecialDay'] +
        alpha4 * df['TrafficLevel'] +
        alpha5 * df['VehicleWeight']
    )
    min_d = df['DemandScore'].min()
    max_d = df['DemandScore'].max()
    df['NormalizedDemand'] = (df['DemandScore'] - min_d) / (max_d - min_d)
    return df


In [13]:
df_model2 = calculate_demand_score(df_model1)
df_model2[['OccupancyRate', 'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleWeight', 'DemandScore', 'NormalizedDemand']].head()


,OccupancyRate,QueueLength,TrafficLevel,IsSpecialDay,VehicleWeight,DemandScore,NormalizedDemand
0,0.105719,1,0.0,0,1.0,0.342288,0.073028
5248,0.197500,2,0.0,0,0.5,0.529000,0.120928
3936,0.384279,2,0.0,0,1.0,0.653712,0.152922
6560,0.513402,2,0.0,0,1.0,0.705361,0.166172
17056,0.319792,2,0.0,0,0.3,0.557917,0.128346


In [14]:
def demand_based_price_model(df, base_price=10, lambda_factor=1.0):
    df = df.copy()
    df['Price'] = base_price * (1 + lambda_factor * df['NormalizedDemand'])
    df['Price'] = df['Price'].clip(lower=0.5 * base_price, upper=2.0 * base_price)

    return df

df_model2 = demand_based_price_model(df_model2)
df_model2[['SystemCodeNumber', 'Timestamp', 'NormalizedDemand', 'Price']].head()


,SystemCodeNumber,Timestamp,NormalizedDemand,Price
0,BHMBCCMKT01,2016-10-04 07:59:00,0.073028,10.730282
5248,BHMNCPHST01,2016-10-04 07:59:00,0.120928,11.209280
3936,BHMMBMMBX01,2016-10-04 07:59:00,0.152922,11.529220
6560,BHMNCPNST01,2016-10-04 07:59:00,0.166172,11.661722
17056,Shopping,2016-10-04 07:59:00,0.128346,11.283464


In [15]:
lot_id = 'BHMBCCMKT01'
sample_day = '2016-10-04'

lot_model1 = df_model1[(df_model1['SystemCodeNumber'] == lot_id) &
                       (df_model1['Timestamp'].dt.date.astype(str) == sample_day)]

lot_model2 = df_model2[(df_model2['SystemCodeNumber'] == lot_id) &
                       (df_model2['Timestamp'].dt.date.astype(str) == sample_day)]


In [16]:
comparison_df = lot_model1[['Timestamp', 'Price']].rename(columns={'Price': 'Model1_Price'}).copy()
comparison_df['Model2_Price'] = lot_model2['Price'].values


In [17]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
output_notebook()


In [18]:
lot_id = 'BHMBCCMKT01'
sample_day = '2016-10-04'

lot_model1 = df_model1[(df_model1['SystemCodeNumber'] == lot_id) &
                       (df_model1['Timestamp'].dt.date.astype(str) == sample_day)]

lot_model2 = df_model2[(df_model2['SystemCodeNumber'] == lot_id) &
                       (df_model2['Timestamp'].dt.date.astype(str) == sample_day)]

bokeh_df = lot_model1[['Timestamp', 'Price']].rename(columns={'Price': 'Model1_Price'}).copy()
bokeh_df['Model2_Price'] = lot_model2['Price'].values


In [19]:
source = ColumnDataSource(bokeh_df)


In [20]:
p = figure(x_axis_type='datetime', title=f'Price Comparison: {lot_id} on {sample_day}',
           width=900, height=400)
p.line(x='Timestamp', y='Model1_Price', source=source,
       line_width=2, color='blue', legend_label='Model 1 - Linear')

p.line(x='Timestamp', y='Model2_Price', source=source,
       line_width=2, color='green', legend_label='Model 2 - Demand-Based')

hover = HoverTool(
    tooltips=[
        ("Time", "@Timestamp{%H:%M}"),
        ("Model 1 Price", "@Model1_Price{$0.00}"),
        ("Model 2 Price", "@Model2_Price{$0.00}")
    ],
    formatters={'@Timestamp': 'datetime'},
    mode='vline'
)
p.add_tools(hover)

p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.toolbar.logo = None
p.toolbar.autohide = True

show(p)


In [21]:
import pathway as pw
from pathway.io.csv import read

In [22]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df_stream = df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'QueueLength',
                'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay']]
df_stream.to_csv("stream_input.csv", index=False)


In [24]:
class InputSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: float
    Capacity: float
    QueueLength: int
    VehicleType: str
    TrafficConditionNearby: str
    IsSpecialDay: int

input_table = read(
    "stream_input.csv",
    schema=InputSchema,
    mode="streaming",
    autocommit_duration_ms=5000,
)


In [25]:
class OutputSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    OccupancyRate: float

@pw.udf
def calculate_occupancy_rate(occupancy, capacity):
    return round(occupancy / capacity, 4)

# Apply UDF to generate new column
output_table = input_table.select(
    SystemCodeNumber=input_table.SystemCodeNumber,
    Timestamp=input_table.Timestamp,
    OccupancyRate=calculate_occupancy_rate(input_table.Occupancy, input_table.Capacity)
)


In [27]:
pw.io.jsonlines.write(output_table, "stream_output.json")


In [29]:
pw.run()


Output()

KeyboardInterrupt: 

In [30]:
df_out = pd.read_json("stream_output.json", lines=True)
df_out.head()

,SystemCodeNumber,Timestamp,OccupancyRate,diff,time
0,BHMEURBRD01,2016-11-14 13:27:00,0.9021,1,1751442156756
1,BHMEURBRD01,2016-11-03 08:26:00,0.4213,1,1751442156756
2,BHMNCPNST01,2016-11-20 10:01:00,0.4412,1,1751442156756
3,BHMBCCMKT01,2016-11-29 16:02:00,0.2218,1,1751442156756
4,Broad Street,2016-12-02 14:58:00,0.8435,1,1751442156756
